In [28]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
import math
from scipy import stats
import geopandas as gpd
%matplotlib inline
from functools import reduce

In [29]:
import importlib
import mr_word_count
importlib.reload(mr_word_count)
from mr_word_count import MRWordFrequencyCount
from mrjob.job import MRJob
import mapreduce as mr

In [30]:
import nltk
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [31]:
#LiveXYZ -- dataframe of XYZ survey data pulled from CSV
LiveXYZ = pd.read_csv('XYZ.csv')

C:\Users\aniha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
len(LiveXYZ)

36192

In [33]:
LiveXYZ.head()

,id,name,address,post_code,latitude,longitude,hours,slug,description,tag_names,...,entrance_method,entrance_origin,entrance_origin_name,floor_,verified,created,space_created,primary_category.0_name,primary_category.1_name,Bars/Dine In/Entertainment/Event Spaces
0,573f7e69425fcd0003082bbf,The Lighthouse,NaN,NaN,40.702913,-73.988120,NaN,NaN,NaN,Bar,...,walkUp,space,Manhattan Bridge Archway Plaza,1,2019-09-20T15:11:04-04:00,2016-05-20T17:15:21-04:00,2016-05-20T17:15:21-04:00,Drinks,Bar,X
1,5b2951ffab163700037a3b7e,Knights Of Columbus - Manresa Council,185 Cedar Grove Avenue,10306,40.560151,-74.101642,NaN,NaN,NaN,"Members Only Club, Rental Space",...,fixed,street,NaN,1,2018-09-07T18:30:17-04:00,2018-06-19T14:57:02-04:00,2018-06-19T14:56:27-04:00,Groups,Organization (Non-Gov't),X
2,598f3d1f08e10a00044a3186,Whitney Grill Deli & Grocery,"86-24, Whitney Avenue",11373,40.742008,-73.878337,NaN,NaN,NaN,"Convenience Store, Deli",...,fixed,street,NaN,1,2019-08-15T13:41:24-04:00,2017-08-12T13:38:39-04:00,2017-08-12T13:38:39-04:00,Essentials,Groceries & Convenience,NaN
3,564392e8a523d000030017f4,Caffè Dante,79-81 Macdougal St,10012,40.728844,-74.001578,Sunday 10:00am-2:00am;Monday 10:00am-2:00am;Tu...,Aperitivo bar celebrating the Italian heritage...,Dante has been a beacon for the community of G...,"Cafe, Cocktail Bar",...,fixed,street,NaN,1,2019-10-30T16:49:16-04:00,2015-11-11T14:11:36-05:00,2015-11-11T14:11:36-05:00,Food,Restaurant,X
4,5c40b086ce48390001d623b2,KoKo Y,"234 Butler St, 2nd Fl.",NaN,40.681683,-73.986067,NaN,A youth-driven and youth-organized project on ...,"Nestled comfortably between birth and death, y...",Art Gallery,...,fixed,space,OpenSource Gowanus,2,2019-08-02T15:48:56-04:00,2019-01-17T11:42:45-05:00,2019-01-17T11:41:14-05:00,Arts & Culture,Art Gallery,NaN


In [34]:
LiveXYZ = LiveXYZ[LiveXYZ['description'].notna()]
len(LiveXYZ)

8586

In [35]:
LiveXYZ = LiveXYZ[LiveXYZ['post_code'].notna()]

LiveXYZ.rename(columns={"post_code": "ZIPCODE"})
LiveXYZ['ZIPCODE'] = LiveXYZ['post_code'].str[:5]

LiveXYZ = LiveXYZ[LiveXYZ['ZIPCODE'].notna()]

#convert the zip code values to int so they can be filtered
LiveXYZ['ZIPCODE'] = LiveXYZ['ZIPCODE'].astype(int)

LiveXYZ = LiveXYZ[(LiveXYZ['ZIPCODE'] >= 10001) & (LiveXYZ['ZIPCODE'] <= 11697)]

LiveXYZ['description'] = LiveXYZ['description'].astype(str)

In [36]:
len(LiveXYZ)

7601

In [37]:
#XYZ_zip df is just the zip code / and primary categories 0 and 1
XYZ_description = LiveXYZ[['ZIPCODE', 'description']]

In [38]:
len(XYZ_description)

7601

In [39]:
XYZ_description.head()

,ZIPCODE,description
3,10012,Dante has been a beacon for the community of G...
7,10001,In 1975 Kolker developed a keen and hands on u...
8,10007,Cloud Gallery is a fine art gallery located in...
9,10019,Whether you’re traveling for business or explo...
11,11229,Our mouthwatering dishes will definitely offer...


In [40]:
pertinent_zips = [11102, 11103, 11105, 11106, 11101, 11104, 11109, 11120, 11206, 11211, 11249, 11207, 11221, 11237, 
                 10026, 10027, 10037, 10030, 10039, 10001, 10011, 10018, 10019, 10020, 10036, 10002, 10034, 10040,
                 10454, 10455, 10459, 10474, 11205, 11216, 11221, 11233, 11238, 11385, 11386, 10109, 11373,
                 11379, 11372, 11354, 11355, 11358, 11222]

In [65]:
len(pertinent_zips)

47

In [41]:
FilteredByZip = XYZ_description[XYZ_description['ZIPCODE'].isin(pertinent_zips)]

#UHF_df = UHF_df[UHF_df['name'].isin(NameVals_to_keep_CD)]

In [42]:
len(FilteredByZip)

2988

In [53]:
#dictionary to map zips to neighborhoods
zip_dict ={ 11101:'Astoria/LIC', 11102:'Astoria/LIC', 11103:'Astoria/LIC', 11105:'Astoria/LIC', 11106:'Astoria/LIC', 11104:'LIC',
           11109:'LIC', 11120:'LIC', 11206:'Williamsburg/Bushwick/Bed-Stuy', 11211:'Williamsburg', 11249:'Williamsburg', 11207:'Bushwick',
           11221:'Bed-Stuy/Bushwick', 11237:'Bushwick', 10026:'Harlem', 10027:'Harlem', 10037:'Harlem', 10030:'Harlem', 10039:'Harlem',
           10001:'Chelsea', 10011:'Chelsea', 10018:'Chelsea', 10019:'Chelsea', 10020:'Chelsea', 10036:'Chelsea', 10002:'LES',
           10034: 'Inwood', 10040: 'Inwood', 10454: 'South Bronx', 10455: 'South Bronx', 10459: 'South Bronx', 10474: 'South Bronx', 
           11205: 'Bed-Stuy', 11216: 'Bed-Stuy', 11233: 'Bed-Stuy', 11238: 'Bed-Stuy',
           11385: 'Ridgewood', 11386: 'Ridgewood', 10109: 'Times Square', 
           11373: 'Elmhurst', 11379: 'Elmhurst', 11372: 'Jackson Heights', 
           11354: 'Flushing', 11355: 'Flushing', 11358: 'Flushing', 11222: 'Greenpoint'}

In [54]:
FilteredByZip['Neighborhood']= FilteredByZip['ZIPCODE'].map(zip_dict)

C:\Users\aniha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
FilteredByZip.head()

43

In [56]:
#Total Number of venues in each neighborhood
df1 = pd.crosstab(FilteredByZip['Neighborhood'], 'count')
df1

col_0,count
Neighborhood,
Astoria/LIC,311
Bed-Stuy,216
Bed-Stuy/Bushwick,24
Bushwick,57
Chelsea,1385
Elmhurst,30
Flushing,84
Greenpoint,77
Harlem,100


In [57]:
len(FilteredByZip)

2988

In [58]:
neighborhood_desc = FilteredByZip
neighborhood_desc = neighborhood_desc.dropna()
len(neighborhood_desc)

2988

In [59]:
neighborhood_desc.head()

,ZIPCODE,description,Neighborhood
7,10001,In 1975 Kolker developed a keen and hands on u...,Chelsea
9,10019,Whether you’re traveling for business or explo...,Chelsea
35,10019,Bringing authentic Italian fare to the heart o...,Chelsea
56,11205,A program that will send absolutely nothing t...,Bed-Stuy
69,10002,A new dimension and home to adventurous artist...,LES


In [60]:
neighborhood_desc['description'] = neighborhood_desc['description'].astype(str)

In [61]:
neighborhood_desc.Neighborhood.unique()

array(['Chelsea', 'Bed-Stuy', 'LES', 'Williamsburg', 'Astoria/LIC',
       'Greenpoint', 'Harlem', 'Ridgewood', 'LIC',
       'Williamsburg/Bushwick/Bed-Stuy', 'South Bronx', 'Elmhurst',
       'Bushwick', 'Jackson Heights', 'Flushing', 'Inwood',
       'Bed-Stuy/Bushwick'], dtype=object)

In [ ]:
# len(neighborhood_desc)

In [62]:
neighborhood_desc.to_csv ('SelectNeighborhoodsDescription.csv', index = False, header=True)

In [ ]:
# XYZ_description['description'] = XYZ_description['description'].str.lower()
# XYZ_description.head()

In [ ]:
#x = XYZ_description['description'].str.cat()

In [ ]:
#(x)

In [ ]:
# class MRWordFrequencyCount(MRJob):
    
#     def mapper(self, _, line):
#         for word in line.split():
#             yield word.lower(),1
        

#     def reducer(self, key, values):
#         yield key, sum(values)

# job = MRWordFrequencyCount(args=[])

# list(mr.runJob(enumerate(x), job))

In [ ]:
# df = pd.DataFrame.from_dict(wordCount, orient='index').reset_index()


# df = df.rename(columns={"index": "word", 0:'count'})
# df = df.sort_values('count', ascending=False)

In [ ]:
# # # rename post_code as ZIPCODE so that it can merge with the shape file
# df = df.rename(columns={"index": "word", 0:'count'})
# df = df.sort_vdf.drop( df[ df['count'] == 1 ].index , inplace=True)
# len(df)alues('count', ascending=False)

In [ ]:
# df.drop( df[ df['count'] == 1 ].index , inplace=True)
# len(df)

In [ ]:
# df.drop( df[ df['count'] == 2 ].index , inplace=True)
# len(df)

In [ ]:
# def remove_punctuation(text):
#     no_punct = ''.join([c for c in text if c not in string.punctuation])
#     return no_punct

In [ ]:
# XYZ_description['description'] = XYZ_description['description'].apply(lambda x:remove_punctuation)
# XYZ_description['description'].unique()

In [ ]:

nltk.download()

In [ ]:
# https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f

In [ ]:
#x = x.translate(str.maketrans('', '', string.punctuation))

In [ ]:
tokenizer = RegexpTokenizer('\s+', gaps=True)

In [ ]:
#tokenizer = RegexpTokenizer(r'\w')

In [ ]:
FilteredByZip['description'] = FilteredByZip['description'].apply(lambda x: tokenizer.tokenize(x.lower()))
FilteredByZip['description'].head(10)

In [ ]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [ ]:
FilteredByZip['description'] = FilteredByZip['description'].apply(lambda x: remove_stopwords(x))
FilteredByZip.head()

In [ ]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [ ]:
FilteredByZip['description'].apply(lambda x: word_lemmatizer(x))

In [ ]:
#stemmer_df = XYZ_description

In [ ]:
#stemmer = PorterStemmer()

In [ ]:
# def word_stemmer(text):
#     stem_text = ''.join([stemmer.stem(i) for i in text])
#     return stem_text

In [ ]:
#stemmer_df['description'] = stemmer_df['description'].apply(lambda x: word_stemmer(x))

In [ ]:
#stemmer_df['description']

In [ ]:
#(XYZ_description['description'])

In [ ]:
# x = list(XYZ_description['description'])

In [ ]:
FilteredByZip.head()

In [ ]:
FilteredByZip['Neighborhood'].unique()

In [ ]:
y = FilteredByZip
y['ZIPCODE'] = y['ZIPCODE'].astype(int)
y['description'] = y['description'].astype(str)

In [ ]:
y.columns

In [ ]:
x = y['description'].str.cat()

In [ ]:
len(x)

In [ ]:
from collections import Counter

words = x.split()
wordCount = Counter(words)



In [ ]:
df = pd.DataFrame.from_dict(wordCount, orient='index').reset_index()


df = df.rename(columns={"index": "word", 0:'count'})
df = df.sort_values('count', ascending=False)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df['word'] = df['word'].str.replace(r'[^\w\s]+', '')

In [ ]:
df.head()

In [ ]:
df = df[df.word != '']

In [ ]:
len(df)

In [ ]:
type(df)

In [ ]:
df.drop( df[ df['count'] == 1 ].index , inplace=True)
len(df)

In [ ]:
df.drop( df[ df['count'] == 2 ].index , inplace=True)
len(df)

In [ ]:
df100 = df.head(150)

In [ ]:
df100

In [ ]:
#df100.to_csv ('Description_dataframe.csv', index = False, header=True)

In [ ]:
word_filter = ['best','art','experience','quality','gallery','space','cuisine', 
               'local','first','unique','traditional','street','authentic','home'
               'heart','variety','neighborhood','artists','modern','classic',
               'world','special','contemporary','create','free','taste','location'
               'community','center','love','finest','small','signature','culinary'
               'famous','atmosphere','feel','inspired','old','music','seasonal','museum']

In [ ]:
dante = ['dante']

In [ ]:
res = any(ele in y['description'] for ele in dante)

In [ ]:
y['description'].dtypes

In [ ]:
zip_code=gpd.read_file("ZIP_CODE_040114.shp")

In [ ]:
zip_code.head()

In [ ]:
#len((zip_code['ZIPCODE']))

In [ ]:
zip_code['ZIPCODE'] = zip_code['ZIPCODE'].astype(int)

In [ ]:
zip_code['ZIPCODE'].dtypes

In [ ]:
zip_code['ZIPCODE'].max()

In [ ]:
zip_code.sort_values('ZIPCODE')

In [ ]:
XYZ_map = zip_code.merge(df1, on='ZIPCODE')

In [ ]:
# XYZ_map = zip_code.merge(df1, left_on = 'ZIPCODE', right_index = True)

In [ ]:
XYZ_map.sort_values('ZIPCODE')

In [ ]:
duplicateRowsDF = XYZ_map[XYZ_map.duplicated(['ZIPCODE'])]
duplicate_rows = duplicateRowsDF.index.tolist()
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicate_rows)

In [ ]:
#drop_list = [15, 18, 19, 50, 65, 106, 107, 108, 118, 134, 135, 136]
UniqueZip = XYZ_map.drop(duplicate_rows)

In [ ]:
#zip_code['POPULATION'].value_counts()

In [ ]:
XYZ_map.head()

In [ ]:
XYZ_map.columns

In [ ]:
XYZ_map['Arts & Culture'].mean()

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
ax.set_title("EDA", fontsize=18)
XYZ_map.plot(axes=ax, column='Food', legend = True, cmap='viridis')

In [ ]:
culture_list = ['Arts & Culture', 'Body', 'Drinks', 'Entertainment',
       'Fashion', 'Features', 'Fitness', 'Food', 'Groups',
       'Home & Hobby', 'Misc', 'Parks & Rec',
       'CULTURE']
for x in culture_list:
    f, ax = plt.subplots(1, figsize=(12, 12))
    ax.set_title("EDA for {}".format(x), fontsize=18)
    UniqueZip.plot(axes=ax, column=x, legend = True, cmap='viridis')

In [ ]:
f, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
ax1 = UniqueZip.plot(ax=ax1, column = 'Food', legend=True).title.set_text('Food')
ax2 = UniqueZip.plot(ax=ax2, column = 'Arts & Culture', legend=True).title.set_text('Arts & Culture')
ax3 = UniqueZip.plot(ax=ax3, column = 'Drinks', legend=True).title.set_text('Drinks')
#ax4 = UniqueZip.plot(ax=ax4, column = 'Entertainment', legend=True).title.set_text('Entertainment')

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
ax.set_title("EDA", fontsize=18)
UniqueZip.plot(axes=ax, column='CULTURE', legend = True, cmap='viridis')

In [ ]:
y = UniqueZip.groupby('COUNTY', group_keys=False).apply(lambda x: x.nlargest(5, "CULTURE"))
y